In [1]:
#dataset link: https://drive.google.com/drive/folders/1n67sVTTzye4jtLfk8n-sa2fH2gTx5Ywt?usp=share_link

In [2]:
import tensorflow as tf
import os
from keras_preprocessing.image import ImageDataGenerator
DATA_DIR = "./Dataset Korosi/"

In [3]:
training_datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2
    )

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    
)

train_generator = training_datagen.flow_from_directory(DATA_DIR, batch_size=32,class_mode='categorical',target_size=(150,150))   
val_generator = train_generator = training_datagen.flow_from_directory(DATA_DIR, subset='validation', batch_size=32,class_mode='categorical',target_size=(150,150))   

Found 304 images belonging to 3 classes.
Found 60 images belonging to 3 classes.


In [5]:
## resep yang nemu di: https://www.kaggle.com/code/vortexkol/alexnet-cnn-architecture-on-tensorflow-beginner

# model_alexnet = tf.keras.models.Sequential([
#     tf.keras.layers.Conv2D(filters=128, kernel_size=(11,11), strides=(4,4), activation='relu', input_shape=(150,150,3)),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2)),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(5,5), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(3,3)),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),

#     tf.keras.layers.Conv2D(filters=256, kernel_size=(1,1), strides=(1,1), activation='relu', padding="same"),
#     tf.keras.layers.BatchNormalization(),
#     tf.keras.layers.MaxPool2D(pool_size=(2,2)),

#     tf.keras.layers.Flatten(),

#     tf.keras.layers.Dense(1024,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(1024,activation='relu'),
#     tf.keras.layers.Dropout(0.5),
#     tf.keras.layers.Dense(3,activation='softmax')  
# ])

## resep yang nemu di: https://medium.com/swlh/alexnet-with-tensorflow-46f366559ce8

model_alexnet = tf.keras.models.Sequential()
model_alexnet.add(tf.keras.layers.experimental.preprocessing.Resizing(150, 150, interpolation="bilinear", input_shape=(150,150,3)))
model_alexnet.add(tf.keras.layers.Conv2D(96, 11, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Lambda(tf.nn.local_response_normalization))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.MaxPooling2D(3, strides=2))
model_alexnet.add(tf.keras.layers.Conv2D(256, 5, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Lambda(tf.nn.local_response_normalization))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.MaxPooling2D(3, strides=2))
model_alexnet.add(tf.keras.layers.Conv2D(384, 3, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.Conv2D(384, 3, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.Conv2D(256, 3, strides=4, padding='same'))
model_alexnet.add(tf.keras.layers.Activation('relu'))
model_alexnet.add(tf.keras.layers.Flatten())
model_alexnet.add(tf.keras.layers.Dense(4096, activation='relu'))
model_alexnet.add(tf.keras.layers.Dropout(0.5))
model_alexnet.add(tf.keras.layers.Dense(4096, activation='relu'))
model_alexnet.add(tf.keras.layers.Dropout(0.5))
model_alexnet.add(tf.keras.layers.Dense(3, activation='softmax'))


In [6]:
model_alexnet.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resizing (Resizing)         (None, 150, 150, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 38, 38, 96)        34944     
                                                                 
 lambda (Lambda)             (None, 38, 38, 96)        0         
                                                                 
 activation (Activation)     (None, 38, 38, 96)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 18, 18, 96)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 5, 256)         614656    
                                                      

In [7]:
import matplotlib.pyplot as plt

In [8]:
f = open("alexnet.csv", "w")
f.write("epochs,loss,val_loss,acc,val_acc")
f.close()


In [10]:
Wsave = model_alexnet.get_weights()
for times in range(10):
    epo = (times+1)*10
    model_alexnet.set_weights(Wsave)
    model_alexnet.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['acc'])
    history = model_alexnet.fit(train_generator,validation_data=val_generator,epochs=epo, verbose = 0)

    f = open("alexnet.csv", "a")
    f.write("\n"+str(epo)+","+
        str(history.history['loss'][-1])+","+
        str(history.history['val_loss'][-1])+","+
        str(history.history['acc'][-1])+","+
        str(history.history['val_acc'][-1])
    )
    f.close()

    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Loss Model')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./alexnet/loss_'+str(epo)+'.png')
    plt.cla()
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.savefig('./alexnet/acc_'+str(epo)+'.png')
    
    plt.clf()

<Figure size 432x288 with 0 Axes>